In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10000)


import altair as alt
alt.data_transformers.disable_max_rows()

import glob
import datetime
import seaborn as sns
import matplotlib.pyplot as pltki

import act
import pyart
import xarray as xr
import os

In [3]:
username = os.getenv("ARM_USERNAME")
token = os.getenv("ARM_TOKEN")

# Identify datastreams

In [4]:
# ppi:  Single-pass full-360o plan position indicator scan 
# ppi2: Single- or multi-pass full or limited sector plan position indicator scan 
# rhi:  Single-pass full-180o range height indicator scan 
# rhi2: Single- or multi-pass full or limited sector range height indicator scan
ds_dl_ppi = 'gucdlppiM1.b1'
ds_dl_rhi = 'gucdlrhiM1.b1'
ds_dl_rhi2 = 'gucdlrhi2M1.b1'

startdate = '2022-10-31'
enddate = '2022-11-04'

# Download datasets

In [ ]:


act.discovery.download_data(username, token, 'dlprofwind4news', startdate, enddate)

In [ ]:
act.discovery.download_data(username, token, ds_dl_ppi, startdate, enddate)
act.discovery.download_data(username, token, ds_dl_rhi, startdate, enddate)
act.discovery.download_data(username, token, ds_dl_rhi2, startdate, enddate)

In [5]:
dl_ppi_files = glob.glob(''.join(['./',ds_dl_ppi,'/*cdf']))
dl_rhi_files = glob.glob(''.join(['./',ds_dl_rhi,'/*cdf']))
dl_rhi2_files = glob.glob(''.join(['./',ds_dl_rhi2,'/*cdf']))

In [6]:
dl_rhi = act.io.armfiles.read_netcdf(dl_rhi_files)

In [7]:
dl_ppi = act.io.armfiles.read_netcdf(dl_ppi_files)


Reading RHI2 scans with act library fails because there are no RHI2 scans

In [8]:
dl_rhi2 = act.io.armfiles.read_netcdf(dl_rhi2_files)

OSError: no files to open

# Examine RHI data

In [9]:
src_rhi = dl_rhi.to_dataframe().reset_index()

In [10]:
(
    len(src_rhi['time'].unique()), 
    len(src_rhi['range'].unique()), 
    len(src_rhi['elevation'].unique()), 
    len(src_rhi['azimuth'].unique())
)

(34837, 400, 13229, 2)

In [17]:
(
    # len(src_rhi['elevation'].unique()), 
    len(src_rhi['azimuth'].unique())
)

2

43561 timestamps over multiple days  (based on our query)

400 discrete range values

13866 discrete elevation angles

2 azimuth directions (0, 270)

In [ ]:
(
    src_rhi['time'].unique(), 
    src_rhi['range'].unique(), 
    src_rhi['elevation'].unique(), 
    src_rhi['azimuth'].unique()
)

## RHI: convert polar coordinates to rectangular coords with the radar at (0,0)

In [ ]:
src_rhi['x'] = src_rhi['range']*np.cos(np.deg2rad(src_rhi['elevation']))
src_rhi['z'] = src_rhi['range']*np.sin(np.deg2rad(src_rhi['elevation']))

## Plot hourly RHI scans (azimuth == 0)

In [ ]:
src_rhi['date'] = src_rhi['time'].dt.date

In [ ]:
src = src_rhi[src_rhi['azimuth'] == 0]

plot_hours = [0,4,8,12,16,20]
plot_days = sorted(src_rhi['date'].unique())

fig, axes = plt.subplots(
    len(src_rhi['date'].unique()), 
    len(plot_hours), 
    figsize=(20,10), 
    sharex=True, sharey=True
)

for i_day, day in enumerate(plot_days):
    for i,hr in enumerate(plot_hours):
        local_src = src[ (src['date'] == day) & (src['time'].dt.hour == hr)]
        ax = axes[i_day][i]
        hexplot = ax.hexbin(local_src['x'], local_src['z'], C=local_src['radial_velocity'], cmap='RdYlBu')
        ax.annotate(str(hr), xy=(-12000, 11000))
    plt.tight_layout()
    plt.colorbar(hexplot)

## Plot hourly RHI scans (azimuth == 270)

In [ ]:
src = src_rhi[src_rhi['azimuth'] == 270]

plot_hours = [0,4,8,12,16,20]
plot_days = sorted(src_rhi['date'].unique())

fig, axes = plt.subplots(
    len(src_rhi['date'].unique()), 
    len(plot_hours), 
    figsize=(20,10), 
    sharex=True, sharey=True
)

for i_day, day in enumerate(plot_days):
    for i,hr in enumerate(plot_hours):
        local_src = src[ (src['date'] == day) & (src['time'].dt.hour == hr)]
        ax = axes[i_day][i]
        hexplot = ax.hexbin(local_src['x'], local_src['z'], C=local_src['radial_velocity'], cmap='RdYlBu')
        ax.annotate(str(hr), xy=(-12000, 11000))
    plt.tight_layout()
    plt.colorbar(hexplot)

# Examine PPI data

## By using xarray DIRECTLY

In [ ]:
dl_ppi_files[:5], dl_ppi_files[-5:]

In [ ]:
df = xr.open_dataset(dl_ppi_files[0]).to_dataframe().reset_index()

In [ ]:
dl_ppi.time.max(), dl_ppi.time.min()

## By using the ACT library to compute vertical wind profiles

In [ ]:
len(dl_ppi[dict(range=slice(0, 2))].to_dataframe()), len(dl_ppi.to_dataframe())

### Limit range values so that computing winds doesn't take so long

In [ ]:
dl_ppi_slice = dl_ppi[dict(range=slice(0, 4))].sel(
    time=slice(datetime.datetime(2022,11,2,4), datetime.datetime(2022,11,2,16))
)

In [ ]:
# Calculate the winds from the gucdlppi dataset.
wind_obj_sliced = act.retrievals.compute_winds_from_ppi(
    dl_ppi_slice,
    remove_all_missing=True, 
    snr_threshold=0.002
)

In [ ]:
wind_obj_sliced.to_dataframe()

In [ ]:
alt.Chart(
    wind_obj_sliced.to_dataframe().reset_index()
).mark_point(shape="wedge", filled=True).encode(
    alt.X('time:T'),
    alt.Y('height:Q'),
    # color=alt.Color(
    #     "wind direction", scale=alt.Scale(domain=[0, 360], scheme="rainbow"), legend=None
    # ),
    angle=alt.Angle("wind_direction", scale=alt.Scale(domain=[0, 360], range=[180, 540])),
    size=alt.Size("wind_speed", scale=alt.Scale(rangeMax=2000, rangeMin=250), title='Wind Speed (m/s)'),
).properties(
    width=1000,
    height=200
)

In [ ]:
(
    alt.Chart(
        wind_obj_sliced.to_dataframe().reset_index()
    ).mark_line().encode(
        alt.X('time:T'),
        alt.Y('wind_direction:Q', title='Wind Direction'),
        alt.Color('height:N')
    ).properties(width=1000, height=200)
&
    alt.Chart(
        wind_obj_sliced.to_dataframe().reset_index()
    ).transform_window(
        rolling_mean='mean(wind_direction)',
        frame=[-4, 4],
        groupby=['height']
    ).mark_line().encode(
        alt.X('time:T'),
        alt.Y('rolling_mean:Q'),
        alt.Color('height:N')
    ).properties(width=1000, height=200)
)

### For all range values

In [ ]:
# Calculate the winds from the gucdlppi dataset.
wind_obj = act.retrievals.compute_winds_from_ppi(
    dl_ppi, remove_all_missing=True, snr_threshold=0.002)

In [ ]:
ppi_calc_winds_df = wind_obj.to_dataframe()

In [ ]:
src = ppi_calc_winds_df.query("height < 100")

In [ ]:
alt.Chart(src.reset_index()).transform_window(
    rolling_mean='mean(wind_direction)',
    frame=[-4, 4],
    groupby=['height']
).mark_line().encode(
    alt.X('time:T'),
    alt.Y('rolling_mean:Q'),
    alt.Color('height:N')
).properties(width=1000)


In [ ]:
alt.Chart(src.reset_index()).mark_line().encode(
    alt.X('time:T'),
    alt.Y('wind_direction'),
    alt.Color('height:N')
).properties(width=1400)

In [ ]:
# Create a display object.
display = act.plotting.TimeSeriesDisplay(
    {"GUC DLPPI Computed Winds over KAZR": wind_obj,
     }, figsize=(20, 10))

display.plot_barbs_from_spd_dir('wind_speed', 'wind_direction',
                                dsname='GUC DLPPI Computed Winds over KAZR',
                                invert_y_axis=False)

# Update the x-limits to make sure both wind profiles are shown
display.axes[0].set_xlim([np.datetime64('2022-08-01T22:10'), np.datetime64('2022-08-01T23:50')])
plt.show()

In [ ]:
(
    len(ppi_calc_winds_df['time'].unique()), 
    len(ppi_calc_winds_df['height'].unique()), 
),(
    ppi_calc_winds_df['time'].unique(), 
    ppi_calc_winds_df['height'].unique(), 
)

8 timestamps over ~36 seconds, every ~5 seconds

400 discrete range values

1 elevation angle

8 azimuth directions (259.1, 304.1, 349.1,  34.100006,  79.100006, 124.100006, 169.09998 , 214.09998)

In [ ]:
(
    len(df['time'].unique()), 
    len(df['range'].unique()), 
    len(df['elevation'].unique()), 
    len(df['azimuth'].unique())
),(
    df['time'].unique(), 
    df['range'].unique(), 
    df['elevation'].unique(), 
    df['azimuth'].unique()
)

In [ ]:
df['azimuth'].unique()

# Look at XPrecip radar data, PPI

In [ ]:
df2 = xr.open_dataset('/Users/elischwat/Downloads/gucxprecipradarS2.00.20221030.000005.raw.nc/gucxprecipradarS2.00.20221030.000130.raw.csu.sail-20221030-000130_702887_22_1_PPI.nc').to_dataframe()

from 

https://cookbooks.projectpythia.org/radar-cookbook/notebooks/foundations/pyart-gridding.html

In [ ]:
radar = pyart.io.read('/Users/elischwat/Downloads/gucxprecipradarS2.00.20221030.000005.raw.nc/gucxprecipradarS2.00.20221030.000130.raw.csu.sail-20221030-000130_702887_22_1_PPI.nc')
print(radar.scan_type)

In [ ]:
fig = plt.figure(figsize=[10, 10])
display = pyart.graph.RadarMapDisplay(radar)
radar.fields.keys()

In [ ]:
display.plot_ppi('DBZ')

# Look at XPrecip radar data, RHI

In [ ]:
# Explore files

radar = pyart.io.read('/Users/elischwat/Downloads/gucxprecipradarS2.00.20221030.000005.raw.nc/gucxprecipradarS2.00.20221030.000005.raw.csu.sail-20221030-000005_702875_22_326_RHI.nc')
print(radar.scan_type)

fig = plt.figure(figsize=[10, 10])
display = pyart.graph.RadarMapDisplay(radar)
radar.fields.keys()

In [ ]:
display.plot_rhi('DBZ')